In [176]:
import pandas as pd
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity

In [162]:
df = pd.read_csv("mbti_dataset.csv",encoding="utf-8")

In [163]:
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [164]:
types = df["type"].unique()
for idx,mbti in enumerate(types):
    df["type"]= df["type"].replace(mbti,idx)

In [165]:
#print(df.loc[0,"posts"])

In [166]:
stop_words = stopwords.words("english")
#create an object of class PorterStemmer
porter = PorterStemmer()

In [167]:
def cleanData(posts):
    
    #remove all hyperlinks
    clean_text = re.sub(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})','',posts)
    #for i in types:
       # clean_text = re.sub(i,"",clean_text,flags=re.IGNORECASE)
    word_list = word_tokenize(clean_text)
    clean_posts = []
    for word in word_list:
        if word.isalpha() and word not in stop_words:
            word = porter.stem(word.lower())
            clean_posts.append(word)
    return " ".join(clean_posts)



In [168]:
df["posts"]= df["posts"].apply(cleanData)

In [169]:
df.head(10)
print(df.iloc[0]["posts"])

intj moment sportscent top ten play experi life on repeat perc experi immers last thing infj friend post facebook commit suicid next day rest sorri hear distress it natur relationship perfect time everi moment exist tri figur hard time time growth game set wellbrutin least thirti minut move leg i mean move sit desk chair weed moder mayb tri edibl healthier altern come three item determin type whichev type want would like use given type cognit function whatnot left thing moder sim inde video game good one note good one somewhat subject i complet promot death given sim enfp what favorit video game grow current favorit video game cool appear late sad someon i thought confid good cherish time solitud revel within inner world wherea time workin enjoy time do worri peopl alway around entp ladi complimentari person well main social outlet xbox live convers even verbal fatigu i realli dig part thread requir high backyard roast eat marshmellow backyard convers someth intellectu follow massag ma

In [170]:
X=df["posts"]
y=df["type"]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [185]:
cv = CountVectorizer()
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 

X_train_count = tfidf_vectorizer.fit_transform(X_train)
X_test_count = tfidf_vectorizer.transform(X_test)

In [186]:
model = MultinomialNB().fit(X_train_count,y_train)

predictions=model.predict(X_test_count)

acc = metrics.accuracy_score(y_test,predictions)
print("predictions:", predictions)
print("accuracy: ", acc)

predictions: [6 6 6 ... 6 0 6]
accuracy:  0.2069164265129683
